In [1]:
# Relational-GCN as implemented by M. Schlichtkrull, T. N. Kipf, P. Bloem, R. van den Berg, I. Titov, M. Welling, 
# in Modeling Relational Data with Graph Convolutional Networks, 2017
import numpy as np
from multigraph import MultiGraph
import unicodecsv as csv

In [ ]:
#import csv
""" EXAMPLE
add,IS,operator
subtract,IS,operator
multiply,IS,operator
divide,IS,operator
open_closure,IS,operator
close_closure,IS,operator
"""
op_graph = MultiGraph()
#with open('operator_graph.csv', 'r') as csvfile:
n = 45
for i in range(n):
    print '.',
    with open('aifb_csv/aifb_relation_'+str(i)+'.csv', 'r') as csvfile:
        graphreader = csv.reader(csvfile, delimiter=",")
        for row in graphreader:
            op_graph.add_connection(row)
print('\n loaded '+str(op_graph.n_rels)+' relations.')

import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

#save the adjacency matrix as well
save_object(op_graph, 'adj_graph.pkl')
print("Saved graph object to disk")

In [2]:
import pickle
with open('adj_graph.pkl', 'rb') as inobj:
    op_graph = pickle.load(inobj)

In [3]:
num_nodes = op_graph.n_nodes
num_relations = op_graph.n_rels

In [4]:
import scipy.sparse as sp
# Create list of adjacency matrices, transpose, and identity
A = list()
for i in range(num_relations):
    k = op_graph.get_relation_label(i)
    A.append(op_graph.get_adjacency_matrix_k(k))
    A.append(op_graph.get_transpose_adjacency_matrix_k(k))
A.append(sp.identity(num_nodes).tocsr())
support = len(A)

# Normalize adjacency matrices
for i in range(len(A)):
    d = np.array(A[i].sum(1)).flatten()
    d_inv = 1. / d
    d_inv[np.isinf(d_inv)] = 0.
    D_inv = sp.diags(d_inv)
    A[i] = D_inv.dot(A[i]).tocsr()
    
X = sp.csr_matrix(A[0].shape)

/usr/local/anaconda3/envs/embeddings/lib/python2.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in divide
  


In [5]:
import random
from collections import defaultdict
# Get targets for training and testing. 
def get_train_test_labels(graph, filename, train_frac=0.8):
    training_dict = defaultdict(list)
    train_ids = list()
    test_ids = list()
    with open(filename, 'r') as csvfile:
        graphreader = csv.reader(csvfile)
        i = 0 #first row is the header, skip it
        for row in graphreader:
            if i == 0:
                i += 1
                continue
            #replace name of node with node index in the graph
            idx = graph.nodes[row[0]]
            training_dict[row[1]].append(idx)
            #randomly sort sample into the training or test set
            if random.random() < train_frac:
                train_ids.append(idx)
            else:
                test_ids.append(idx)
    num_labels = len(training_dict.keys())
    #create a sparse matrix with all the nodes that have labels
    labels = sp.lil_matrix((num_nodes, num_labels))
    i = 0
    for k,v in training_dict.iteritems():   
        for s in v:
            labels[s, i] = 1
        i += 1
    return (labels, train_ids, test_ids)   

In [6]:
from rgcn.utils import *
#get train, validation, and testing sets
VALIDATION = True

y, train_idx, test_idx = get_train_test_labels(op_graph, 'aifb_csv/completeDataset.csv', train_frac=0.8)
y_train, y_val, y_test, idx_train, idx_val, idx_test = get_splits(y, train_idx, test_idx,VALIDATION)

train_mask = sample_mask(idx_train, y.shape[0])

In [7]:
import keras
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

from rgcn.layers.graph import GraphConvolution
from rgcn.layers.input_adj import InputAdj

encoding_dim = 16
L2 = 0.0
LR = 0.01
DO = 0.0

# Define empty dummy feature matrix (input is ignored as we set featureless=True)
# In case features are available, define them here and set featureless=False.
A_in = [InputAdj(sparse=True) for _ in range(support)]
X_in = Input(shape=(X.shape[1],), sparse=True)

# Define model architecture
ConvLayer1 = GraphConvolution(encoding_dim, support=support, num_bases=-1, use_bias=False, featureless=True, activation='relu', kernel_regularizer=l2(L2))
DropOutLayer = Dropout(DO)
ConvLayer2 = GraphConvolution(y_train.shape[1], support=support, use_bias=False, featureless=False, num_bases=-1, activation='softmax')

the_code = ConvLayer1([X_in] + A_in)
H = DropOutLayer(the_code)
Y = ConvLayer2([H] + A_in)

# Compile model
model = Model(inputs=[X_in] + A_in, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=LR))
model.summary()


Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_92 (InputLayer)           (None, 8284)         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_4 (I

In [8]:
print('number of nodes',num_nodes)
print('number of relations',num_relations)

('number of nodes', 8284)
('number of relations', 45)


In [9]:
import time
preds = None
NB_EPOCH = 50
# Fit
for ep in range(1, NB_EPOCH + 1):
    # Log wall-clock time
    t = time.time()
    # Single training iteration
    model.fit([X] + A, y=y_train, batch_size=num_nodes, sample_weight=train_mask, epochs=1, shuffle=False, verbose=0)
    #model.fit([X] + A, y_train, sample_weight=train_mask,
    #          batch_size=num_nodes, epochs=1, shuffle=False, verbose=1)
    if ep % 1 == 0:
        # Predict on full dataset
        preds = model.predict([X] + A, batch_size=num_nodes)
        # Train / validation scores
        train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                       [idx_train, idx_val])
        print("Epoch: {:04d}".format(ep),
              "train_loss= {:.4f}".format(train_val_loss[0]),
              "train_acc= {:.4f}".format(train_val_acc[0]),
              "val_loss= {:.4f}".format(train_val_loss[1]),
              "val_acc= {:.4f}".format(train_val_acc[1]),
              "time= {:.4f}".format(time.time() - t))

    else:
        print("Epoch: {:04d}".format(ep),
              "time= {:.4f}".format(time.time() - t))

('Epoch: 0001', 'train_loss= 1.3217', 'train_acc= 0.9196', 'val_loss= 1.3347', 'val_acc= 0.9643', 'time= 5.7927')
('Epoch: 0002', 'train_loss= 1.2109', 'train_acc= 0.9196', 'val_loss= 1.2452', 'val_acc= 0.9643', 'time= 1.0161')
('Epoch: 0003', 'train_loss= 1.0579', 'train_acc= 0.9196', 'val_loss= 1.1182', 'val_acc= 0.9643', 'time= 0.9053')
('Epoch: 0004', 'train_loss= 0.8904', 'train_acc= 0.9107', 'val_loss= 0.9723', 'val_acc= 1.0000', 'time= 1.0042')
('Epoch: 0005', 'train_loss= 0.7106', 'train_acc= 0.9286', 'val_loss= 0.8036', 'val_acc= 1.0000', 'time= 0.8632')
('Epoch: 0006', 'train_loss= 0.5493', 'train_acc= 0.9286', 'val_loss= 0.6377', 'val_acc= 1.0000', 'time= 0.8651')
('Epoch: 0007', 'train_loss= 0.4173', 'train_acc= 0.9464', 'val_loss= 0.4892', 'val_acc= 1.0000', 'time= 0.8672')
('Epoch: 0008', 'train_loss= 0.3115', 'train_acc= 0.9554', 'val_loss= 0.3592', 'val_acc= 1.0000', 'time= 0.8735')
('Epoch: 0009', 'train_loss= 0.2343', 'train_acc= 0.9554', 'val_loss= 0.2562', 'val_acc=

In [10]:
# Testing
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print("Test set results:",
      "loss= {:.4f}".format(test_loss[0]),
      "accuracy= {:.4f}".format(test_acc[0]))

('Test set results:', 'loss= 0.0304', 'accuracy= 1.0000')


In [11]:
coding_model = Model(inputs=[X_in] + A_in, outputs=the_code)
embedding = coding_model.predict([X] + A, batch_size=num_nodes)

In [12]:
#save the embeddings in order to plot them later
# serialize model to JSON
model_json = coding_model.to_json()
with open("rgcn_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
coding_model.save_weights("rgcn_model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

#save the adjacency matrix as well
save_object(op_graph, 'adj_graph.pkl')
print("Saved graph object to disk")